In [1]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv('Tripadvisor_crawling.csv', usecols=['names', 'main_category', 'ratings', 'keywords'])
data

,names,main_category,keywords,ratings
0,대포항,랜드마크,즐길거리 & 랜드마크,4.423077
1,타워갤러리,랜드마크,아트갤러리,4.600000
2,방짜유기박물관,랜드마크,전문 박물관,4.250000
3,남산 케이블카,랜드마크,전차,4.358516
4,화순적벽 관광지,랜드마크,자연 & 야생동물 서식지,5.000000
...,...,...,...,...
971,여진불교미술관,교통편,아트갤러리,4.416667
972,커피박물관,교통편,전문 박물관,4.500000
973,평화의 소녀상,교통편,기념비 및 동상,5.000000
974,구천동 계곡,교통편,"국립공원, 자연 & 야생동물 서식지, 계곡",5.000000


In [3]:
data['keywords'][data['keywords'].isna()] = ''    # NaN값을 빈 문자열로 대체

def preprocess(string):
    
# '/', '&' 같은 문자 제거
# '장소', '및', '전문', '시설' 등과 같은 수식어 제거    
# 필요한경우 띄어쓰기 제거 (e.g. 인근 지역 -> 인근지역)

    # 공통
    
    string = string.replace(' /', '')
    string = string.replace(' &', '')
    string = string.replace(' 및', '')
    string = string.replace('아쿠아리움', '수족관') # 중복제거를 위해 아쿠아리움을 수족관으로 통일
    
    # main_category
    
    string = string.replace('보트 투어', '보트투어')
    string = string.replace('수상 스포츠', '수상스포츠')
    string = string.replace('즐길 거리', '즐길거리')
    string = string.replace('야외 활동', '야외활동')
    string = string.replace('여행자 자료', '여행자자료')

    # keywords

    string = string.replace('쇼핑몰', '쇼핑')
    string = string.replace('전문 박물관', '전문박물관 박물관')
    string = string.replace('어린이 박물관', '어린이박물관 박물관')
    string = string.replace('자연사 박물관', '자연사박물관 박물관')
    string = string.replace('군사 박물관', '군사박물관 박물관')
    string = string.replace(' 장소', '') # 종교적인 장소, 교육적인 장소
    string = string.replace('인근 지역', '인근지역')
    string = string.replace('경관이 좋은 산책로', '경관좋은산책로')
    string = string.replace('유서 깊은 산책로', '유서깊은산책로')
    string = string.replace('공공기관 건물', '공공기관건물')
    string = string.replace('고대 유적', '고대유적')
    string = string.replace('군사 기지', '군사기지')
    string = string.replace('자동차 경주장', '자동차경주장')
    string = string.replace('절경 드라이브 코스', '절경드라이브코스')
    string = string.replace('하이킹 트레일', '하이킹트레일')
    string = string.replace('야생동물 서식지', '야생동물서식지')
    string = string.replace('엔터테인먼트 센터', '엔터테인먼트센터')
    string = string.replace('도자기 스튜디오', '도자기스튜디오')
    string = string.replace('방 탈출 게임', '방탈출게임 게임')
    string = string.replace('기타 놀이', '게임') 
    string = string.replace('미니 골프', '미니골프')
    string = string.replace('물건 찾기 게임', '물건찾기게임 게임')
    string = string.replace('테마 파크', '테마파크')
    string = string.replace('대중교통 시스템', '대중교통시스템')
    
    return string

data['main_category'] = data['main_category'].apply(preprocess)
data['keywords'] = data['keywords'].apply(preprocess)





def leave_space(string):
    return ' ' + string

data['keywords_aggregated'] = data['main_category'] + data['keywords'].apply(leave_space)

data = data.drop(['main_category', 'keywords'], axis=1)

data = data.rename(columns = {'keywords_aggregated' : 'keywords'})


# 중복되는 단어 처리

def duplicates_handle(string):
    string_list = string.split(' ')
    
    tmp = []
    
    to_del = []
    
    for val in string_list:
        if string_list.count(val)!=1:
            to_del.append(val)
            tmp.append(val)

    for val in to_del:
        string = string.replace(val, '')
    
    tmp_str = ' '
    
    for val in list(set(tmp)):
        tmp_str = tmp_str + val
    
    string = string + tmp_str
    
    return string


words = ['스포츠', '게임', '박물관', '공원', '쇼핑']

for word in words:
    data['keywords'] = data['keywords'].apply(duplicates_handle)

                   
data

,names,ratings,keywords
0,대포항,4.423077,즐길거리 랜드마크
1,타워갤러리,4.600000,랜드마크 아트갤러리
2,방짜유기박물관,4.250000,랜드마크 전문박물관 박물관
3,남산 케이블카,4.358516,랜드마크 전차
4,화순적벽 관광지,5.000000,랜드마크 자연 야생동물서식지
...,...,...,...
971,여진불교미술관,4.416667,교통편 아트갤러리
972,커피박물관,4.500000,교통편 전문박물관 박물관
973,평화의 소녀상,5.000000,교통편 기념비 동상
974,구천동 계곡,5.000000,"교통편 국립공원, 자연 야생동물서식지, 계곡"


In [4]:
count_vector = CountVectorizer(ngram_range=(1, 10))
c_vector_keywords = count_vector.fit_transform(data['keywords'])
c_vector_keywords.shape

(976, 1207)

In [8]:
count_vector.get_feature_names()

['가든',
 '가든 경관좋은산책로',
 '가든 공원',
 '가든 놀이터',
 '가든 박물관',
 '간헐천',
 '강좌',
 '강좌 워크숍',
 '강좌 워크숍 스포츠',
 '강좌 워크숍 스포츠 캠프',
 '강좌 워크숍 스포츠 캠프 클리닉',
 '건축물',
 '건축물 랜드마크',
 '건축물 브리지',
 '건축물 신성한',
 '건축물 신성한 종교적인',
 '건축물 역사박물관',
 '건축물 유령',
 '건축물 유령 마을',
 '게임',
 '게임 가든',
 '게임 가든 놀이터',
 '게임 공원',
 '게임 공원 놀이터',
 '게임 놀이기구',
 '게임 놀이기구 액티비티',
 '게임 물길',
 '게임 물길 바이크',
 '게임 물길 바이크 트레일',
 '게임 물길 바이크 트레일 공원',
 '게임 물길 바이크 트레일 공원 보기',
 '게임 스포츠',
 '게임 스포츠 컴플렉스',
 '게임 스포츠 컴플렉스 공원',
 '경관좋은산책로',
 '경관좋은산책로 랜드마크',
 '경관좋은산책로 분수대',
 '경관좋은산책로 자연',
 '경관좋은산책로 화산',
 '경기장',
 '경마장',
 '계곡',
 '고대유적',
 '공공기관건물',
 '공원',
 '공원 가든',
 '공원 가든 경관좋은산책로',
 '공원 경관좋은산책로',
 '공원 경마장',
 '공원 계곡',
 '공원 고대유적',
 '공원 공원',
 '공원 공원 놀이터',
 '공원 관광센터',
 '공원 관광센터 즐길거리',
 '공원 관광센터 즐길거리 랜드마크',
 '공원 관측소',
 '공원 관측소 천체투영관',
 '공원 교회',
 '공원 교회 성당',
 '공원 국립공원',
 '공원 극장',
 '공원 기념비',
 '공원 기념비 동상',
 '공원 놀이터',
 '공원 대중교통시스템',
 '공원 대학',
 '공원 대학 학교',
 '공원 동굴',
 '공원 동물원',
 '공원 동물원 수족관',
 '공원 등대',
 '공원 목장',
 '공원 물길',
 '공원 물길 가든',
 '공원 물길 경관좋은산책로',
 '공원 물길 즐길거리',
 '공원 물길 즐길거리

In [9]:
len(count_vector.get_feature_names())

1207

In [5]:
c_vector_keywords.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [6]:
keyword_c_sim = cosine_similarity(c_vector_keywords, c_vector_keywords).argsort()[:, ::-1]
keyword_c_sim.shape

(976, 976)

In [7]:
def get_recommend_trip_list(df, place, numb, top=30):
    '''df --> 데이터프레임
    place --> 비슷한 여행지 찾고싶은 특정 여행지를 string 형식으로 입력
    numb --> 추천받고싶은 여행지 개수'''
    #비슷한 여행지를 찾고 싶은 '특정 여행지'의 인덱스 추출
    target_place_index = df[df['names'] == place].index.values
    #비슷한 코사인 유사도를 가진 여행지들의 인덱스 추출
    sim_index = keyword_c_sim[target_place_index, :top].reshape(-1)
    #자기자신 없애기
    sim_index = sim_index[sim_index != target_place_index]
    #추출한 여행지를 data frame으로 만들고 별점(ratings) 순으로 정렬한 뒤 return
    result = df.iloc[sim_index].sort_values('ratings', ascending=False)[:numb]
    print(len(result))
    return result

# 테스트
get_recommend_trip_list(df=data, place='익산미륵사지', numb=10)

10


,names,ratings,keywords
253,청량사,5.000000,랜드마크 신성한 종교적인
327,충익사,5.000000,랜드마크 신성한 종교적인
12,도림사,5.000000,랜드마크 신성한 종교적인
224,도솔암,5.000000,랜드마크 신성한 종교적인
317,국청사,5.000000,랜드마크 신성한 종교적인
348,백련사,5.000000,랜드마크 신성한 종교적인
854,무상사,4.875000,랜드마크 신성한 종교적인
523,관촉사,4.830000,랜드마크 신성한 종교적인
247,길상사,4.815789,랜드마크 신성한 종교적인
609,상원사,4.810000,랜드마크 신성한 종교적인
